# 4. Working with larger data sets

This workbook uses a large sample dataset in order to more realistically simulate the set up of the analysis process from start to finish. The dataset is not included directly with this code because of its size but can be downloaded from Zenodo - https://zenodo.org/record/3630511#.XjHhJGj7SUl

We assume that you have downloaded the data, unpacked it and placed it in the *example_data* folder before running this notebook.

The sample analysed is a Titanium-64 Alloy (https://en.wikipedia.org/wiki/Ti-6Al-4V). The energy of the imaging beam was 89 KeV meaning a wavelength of ~0.14 Å.

The spectra are sampled at a rate of 10 Hz. There are approximately 5500 frames, corresponding to an experiment approximately 9 minutes long.

The experiment begins at room temperature, heats for approximately 200 seconds, is held at constant temperature for about 100 seconds. The material being analysed then undergoes a mechanical deformation for approximately 10 seconds around 340 seconds from the start of the experiment. After this is sample is then cooled back to room temperature. We expect the crystal structure of the material to be primarily hexagonal ($\alpha$-phase) at room temperature, primarily BCC ($\beta$-phase) and high temperature and we expect some structural change as a result of the deformation.

Although in theory we could follow the evolution of the cubic and hexagonal crystals in the material by fitting only a single peak for each, we want to consider as many of the possible orientations as possible to get better statistics.

We follow the beta structure by fitting peaks corresponding to the BCC Cubic Miller indices: 110, 200, 211, 220 and 310.

We follow the alpha structure by fitting peaks corresponding to the hexagonal indices: 10-10, 0002, 10-11, 10-12, 11-20, 10-13, 11-22, 20-21, 0004 and 20-22.

## 4.1. Calculating peak angles

Bragg's law gives the angle of scattering from a crystal lattice $(\theta)$ as a function of the radiation wavelength planes $(\lambda)$:

$$ \lambda = 2d \sin (\theta) $$

We have incoming radiation $(\theta)$ of wavelength 0.14 Å. The Ti lattice constants are $a$ = 2.95 Å, $c$ = 4.68 Å.

### 4.1.1. Cubic crystal symmetry

For a cubic system the relation between lattice spacing and the lattice constant $(a)$ is:

$$ \frac{1}{d^2} = \frac{h^2 + k^2 + l^2}{a^2}  $$

Combining this relation with Bragg's law gives the relation between the scattering angle, radiation wavelength and lattice constant:

$$ \sin^2(\theta) = \frac{\lambda^2}{4a^2}(h^2 + k^2 + l^2) $$

For BCC the symmetry causes systematic absences in odd numbered hkl planes so we expect to see the 110, 200, 211, 220 and 310 peaks.

The approximate $2\theta$ scattering angles in order of increasing two theta angle are then:

* 110: 3.85°
* 200: 5.44°
* 211: 6.66°
* 220: 7.70°
* 310: 8.60°

### 4.1.2. Hexagonal crystal symmetry

Using the Bravais-Miller system of notation for the hexagonal crystal symmetry (h, k, i, l) the relation between the lattice spacings and the lattice constants is:

$$ d = \frac{a}{\sqrt{\frac{4}{3}(h^2 + k^2 + hk) + \frac{a^2}{c^2}l^2)}} $$

Combining this with Bragg's law gives the relation between the scattering angle, radiation wavelength and lattice constants:

$$ \theta = \sin^{-1} \left( \frac{\lambda \sqrt{\frac{4}{3}(h^2 + k^2 + hk) + \frac{a^2}{c^2}l^2)}}{2a} \right) $$

Iterating through the combinations of indices gives $2\theta$ angles of:

* 10-10: 3.14°
* 0002: 3.43°
* 10-11: 3.58°
* 10-12: 4.65°
* 11-20: 5.44°
* 10-13: 6.03°
* 20-20: 6.28°
* 11-22: 6.43°
* 20-21: 6.51°
* 0004: 6.86°
* 20-22: 7.16°


## 4.2 Finding the peaks

First lets take a look the diffraction pattern and see how it looks.

In [ ]:
%matplotlib inline

from xrdfit.spectrum_fitting import PeakParams, FitSpectrum, FitExperiment

spectral_data = FitSpectrum('../example_data/example_data_large/adc_065_TI64_NDload_900C_15mms_00001.dat', 90)
spectral_data.plot_polar()

Now plot a single spectrum

In [ ]:
spectral_data.plot(1)

Zoom in a bit.

In [ ]:
spectral_data.plot(1, x_range=(2.5, 10.5))

This looks reasonable. Since this is the first spectrum at the start of the experiment we expect high intensity hexagonal peaks and weak beta peaks. Using our calculations from above we can zoom in a section at a time and assign the peaks.

In [ ]:
spectral_data.plot(1, x_range=(3, 4), show_points=True)

These peaks fit with being 10-10, 0002, a weak 110 and 10-11. We should fit 0002, 110 and 10-11 as a triplet due to their proximity.

The 110 is a little way off our calculated value - about 8% lower than expected. This is due to alloying - inclusion of Vanadium in the beta phase. This is a constant percentage offset so we modify our original estimates of the peak positions to:

* 110: 3.55°
* 200: 5.01°
* 211: 6.13°
* 220: 7.08°
* 310: 7.92°

We can quickly plot the peak params to check they are OK, then do the fits and check the fit results.

In [ ]:
peak_params = [PeakParams('(10-10)', (3.02, 3.27)),
               PeakParams('(0002)(110)(10-11)',  (3.3, 3.75), [(3.4, 3.44), (3.52, 3.56), (3.57, 3.61)])]

spectral_data.plot_peak_params(peak_params, 1, show_points=True)

spectral_data.fit_peaks(peak_params, 1)
spectral_data.plot_fit('(10-10)')
spectral_data.plot_fit('(0002)(110)(10-11)')

That looks reasonable. Now on to the next section of the spectrum.

In [ ]:
spectral_data.plot(1, x_range=(4.4, 5.8), show_points=True)

These are in the right positions for 10-12, 200 and 11-20.These area ll distinct enough to be fitted as singlets. Again lets quickly plot the fits to check they are OK.

In [ ]:
peak_params = [PeakParams('(10-12)', (4.54, 4.8)),
               PeakParams('(200)', (4.9, 5.10)),
               PeakParams('(11-20)', (5.35, 5.6))]

spectral_data.fit_peaks(peak_params, 1)
for fit in spectral_data.fitted_peaks:
    fit.plot()

Now moving up to the next section of the spectrum

In [ ]:
spectral_data.plot(1, x_range=(5.75, 7.5), show_points=True)

The first peak is the hexagonal 10-13. Going with our revised estimates for the cubic structures the small peak at 6.1° must be the 211. The next four strong peaks then correspond to 20-20, 11-22, 20-21 and 0004 in ascending order. There is then a weak peak for the cubic 220 and 20-22 is at about 7.2°. We probably dont need the fit for the 20-22 peak as we have got enough hexagonal peaks at this point. However in order to get the 220 peak we still need to include the 20-22 as the peaks are so close that we wont be able to reliably 220 as a singlet.

With the proximity of these peaks, they would be probably be good to fit as a doublet, singlet, doublet and singlet and then doublet.

In [ ]:
peak_params = [PeakParams('(10-13)(211)', (5.9, 6.25), [(6.00, 6.05), (6.10, 6.15)]),
               PeakParams('(20-20)', (6.21, 6.4)),
               PeakParams('(11-22)(20-21)',  (6.37, 6.71), [(6.43, 6.47), (6.52, 6.56)]),
               PeakParams('(0004)',  (6.75, 6.95), [(6.82, 6.87)]),
               PeakParams('(220)(20-22)', (6.95, 7.35), [(7.05, 7.12), (7.16, 7.20)])]

spectral_data.plot_peak_params(peak_params, 1, show_points=True)

spectral_data.fit_peaks(peak_params, 1)
for fit in spectral_data.fitted_peaks:
    fit.plot()

It would be good to try and get another peak for the cubic phase - we should see if we can find the 310 peak.

In [ ]:
spectral_data.plot(1, x_range=(7.3, 8.1), show_points=True)

The first peak is probably 10-14, we will ignore this as we have enough hexagonal peaks and just try to get the very small 310 peak.

In [ ]:
peak_params = [PeakParams('(310)', (7.75, 8.05))]

spectral_data.fit_peaks(peak_params, 1)
for fit in spectral_data.fitted_peaks:
    fit.plot()

It's not a great fit because the peak intensity is so low but when the percentage of beta phase increases later in the experiment the fit should be better.

Lets put all these peak params together and do a time fit.

## 4.3 Time fitting

### 4.3.1. Initial fit attempt

Lets start by fitting these peaks in every 100th frame to check the fits are OK before fitting the full data set.

In this case I choose to reuse the fits when running the fits. Reusing the fits can cause problems if there is a significant difference between frames but can be faster if the fits are similar from frame to frame. It is best to try with and without and see which is quicker!

In this case reusing the fits is quicker because some of the triplet fits have close peaks with very different magnitudes so it is hard for the parameter guessing algorithm to guess good initial parameters to get the inflection point between them.

In [ ]:
frame_time = 0.1
file_string = '../example_data/example_data_large/adc_065_TI64_NDload_900C_15mms_{:05d}.dat'
first_cake_angle = 90
cakes_to_fit = 1
peak_params = [PeakParams('(10-10)', (3.02, 3.27)),
               PeakParams('(0002)(110)(10-11)',  (3.3, 3.75), [(3.4, 3.44), (3.52, 3.56), (3.57, 3.61)]),
               PeakParams('(10-12)', (4.54, 4.8)),
               PeakParams('(200)', (4.9, 5.10)),
               PeakParams('(11-20)', (5.35, 5.6)),
               PeakParams('(10-13)(211)', (5.9, 6.25), [(6.00, 6.05), (6.10, 6.15)]),
               PeakParams('(20-20)', (6.21, 6.4)),
               PeakParams('(11-22)(20-21)',  (6.37, 6.71), [(6.43, 6.47), (6.52, 6.56)]),
               PeakParams('(0004)',  (6.75, 6.95), [(6.82, 6.87)]),
               PeakParams('(220)(20-22)', (6.95, 7.35), [(7.05, 7.12), (7.16, 7.20)]),
               PeakParams('(310)', (7.75, 8.05))
              ]
max_frame = 5657
merge_cakes = False
frames_to_fit = range(1, max_frame, 100)
experiment = FitExperiment(frame_time, file_string,first_cake_angle, cakes_to_fit, peak_params, merge_cakes, frames_to_fit)

experiment.run_analysis(reuse_fits=True)

You may have noticed that the progress bar does not fill linearly. For some time steps the fitting stalls a little. This is because the initial parameters are closer to the final parameters at some time points compared to others. If the fit is complex or some of the maxima intensities are low, many iterations of the fitting minimiser may required to get a good fit. This isn't necessarily a problem but excessive stalling can be indicative of poor initial parameters.

After the fitting algorithm completes, if there are any peak fits which required more than 500 minimisation steps, they are highlighted in a report. These points should be checked in particular to see if there was a problem with the fitting. If you find that this report flags too many false positives you can increase the number of minimisation steps that triggers the report by providing the `evaluation_threshold` parameter to the `run_analysis` function.

In this case there are 5 peaks which the fitting has had trouble with at some point in the fitting process. The (0002)(110)(10-11), (20-20) and (11-22)(20-21) peaks only have 1 or 2 slow fits each so they are probably nothing to worry about. However, the (10-13)(211) and (220)(20-22) peaks have multiple slow fits - these are probably worth checking.

### 4.3.2 Refining the "(10-13)(211)" peak

Lets look at the (10-13)(211) peak first. First we can look at the fit parameters to see if there is anything unusual.

In [ ]:
experiment.plot_fit_parameter("(10-13)(211)", "maximum_1_center", show_points=True)
experiment.plot_fit_parameter("(10-13)(211)", "maximum_1_height", show_points=True)
experiment.plot_fit_parameter("(10-13)(211)", "maximum_2_center", show_points=True)
experiment.plot_fit_parameter("(10-13)(211)", "maximum_2_height", show_points=True)

There is definitely something wrong here. The error bars are all over the place and the parameters dont change smoothly over time. 

If there is no error estimate available for a parameter, the data point on the graph is shown as a triangle. The lack of an error estimate may indicate a problem with the fit but not necessarily. The error estimate is made by inverting the curvature matrix of the fit. This will not work if changing the parameter does not significantly affect the fitting result or if changing the parameter results in an evaluation error.

In this case the fits for both maxima look poor so there is probably something obviously wrong with the fits. We can plot the fits directly to see what they look like.

In [ ]:
experiment.plot_fits(peak_names=["(10-13)(211)"], num_timesteps=20)

We can see that although the 211 peak is present at the beginning of the experiment it disappears entirely on heating and doesnt reappear later in the experiment. The doublet fit is still trying to fit two peaks however even though there is one and this breaks the fit.

In this case it is probably better to ignore the 211 peak entirely as is disappears very quickly and we cant get much information out of it. Since the 211 peak is so small, 10-13 peak should be fine to fit on its own as a singlet.

If we wanted to try and get some more information from the 211 peak it would be best to do this as a separate fit experiment for just this peak and a limited subset of the total frames.

### 4.3.3 Refining the "(220)(20-22)" peak

Lets plot the parameters for the (220)(20-22) peak first: 

In [ ]:
experiment.plot_fit_parameter("(220)(20-22)", "maximum_1_center", show_points=True)
experiment.plot_fit_parameter("(220)(20-22)", "maximum_1_height", show_points=True)
experiment.plot_fit_parameter("(220)(20-22)", "maximum_2_center", show_points=True)
experiment.plot_fit_parameter("(220)(20-22)", "maximum_2_height", show_points=True)

The 220 peak looks fine but it is the 20-22 that is struggling. Lets plot the fits to see why.

In [ ]:
experiment.plot_fits(peak_names=["(220)(20-22)"], num_timesteps = 30)

There are a few things going on here. From frame 5 to 15 there is a small peak which appears to the left of the fit, this likely confuses the baseline slightly. We cant exclude this however because bringing the peak bounds in would result in one of 220 or 20-22 falling off the edge of the fit later in the experiment when they get further apart.

From frame 20 to 35 the 20-22 peak disappears almost completely. This is OK and the peak is picked up again when it reappears about frame 35 but we should note the the errors on these parameters for these frames are likely to be very large.

Also at various points in the fit the intensities of the peaks are very low, with a signal to noise (peak height divided by baseline noise) of only around 1.5. Anywhere where the peak height is not far above the noise the fitting will struggle a little. This is to be expected and just requires treating the resultant parameters with care.

There is nothing as such we can do to improve these fits here. Perhaps if these peaks were important they could be fitted separately in a new `FitExperiment` instance, reducing the number of time steps and adjusting the starting parameters. This might seem a little awkward but `xrdfit` is not really designed to provide high quality fits for peaks with a low signal to noise - it is more designed for high throughput analysis of clearer peaks.

### 4.3.4. Implementing the refinements

Lets remove the 211 peak and try the fit again. Lets also analyse more frames to increase the data resolution for later analysis. As we increase the frequency of frames the average fit time per spectrum will likely decrease slightly if reuse fits is turned on. Since each frame will differ less from the previous frame the final parameters from the previous fit will be closer to the final parameters of the next fit.

In [ ]:
peak_params = [PeakParams('(10-10)', (3.02, 3.27)),
               PeakParams('(0002)(110)(10-11)',  (3.3, 3.75), [(3.4, 3.44), (3.52, 3.56), (3.57, 3.61)]),
               PeakParams('(10-12)', (4.54, 4.8)),
               PeakParams('(200)', (4.9, 5.10)),
               PeakParams('(11-20)', (5.35, 5.6)),
               PeakParams('(10-13)', (5.9, 6.15), [(6.00, 6.05)]),
               PeakParams('(20-20)', (6.21, 6.4)),
               PeakParams('(11-22)(20-21)',  (6.37, 6.71), [(6.43, 6.47), (6.52, 6.56)]),
               PeakParams('(0004)',  (6.75, 6.95), [(6.82, 6.87)]),
               PeakParams('(220)(20-22)', (6.95, 7.35), [(7.05, 7.12), (7.16, 7.20)]),
               PeakParams('(310)', (7.75, 8.05))
              ]
max_frame = 5657
frames_to_fit = range(1, max_frame, 30)
experiment = FitExperiment(frame_time, file_string, first_cake_angle, cakes_to_fit, peak_params, merge_cakes, frames_to_fit)

experiment.run_analysis(reuse_fits=True)

The (10-13) peak is now reporting no fits with many iterations. Notice that the analysis also runs quicker when poor fits are improved. The 200 peak is now having a little trouble however so these fits should likely be examined.

For a more detailed report which specifies which peaks took the longest to fit you can run the fit report again in detailed mode. This can be useful if the fitting is too slow and you want to identify which peak in particular is taking all of the processing time. Each time `FitExperiment.run_analysis` is run, it generates a `FitReport`. This can be printed with the `print` method.

In [ ]:
experiment.fit_report.print(detailed=True)

For the purposes of this tutorial we now say that we are happy with these fits and proceed do some science with the fits.

## 4.4. Doing some science

The whole point of the fitting is to use the fitting parameters as part of an analysis to do some science. Here we will show some brief examples of how an analysis might be done. This is not a complete and thorough analysis of this dataset, but serves to show an example of how the fits can be used.

First lets read in some supplementary experimental data. This is found in the in "example_data/instrument_data/065_ETMT_analogue_output.dat". This data file has direct instrument voltages from sensors during the experiment where the data was taken at the same rate as the spectra. The data columns correspond to frame number, temperature, imposed load and resultant sample deformation. Since the data are voltages we must scale them using the calibration from the sensors to get the real values.

The scaling for the temperature data is 150°C V$^{-1}$, the imposed strain data is 25 N V$^{-1}$ and the resultant deformation is scaled as 0.5 mm V$^{-1}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

instrument_data = np.loadtxt("../example_data/instrument_data/065_ETMT_data.txt", skiprows=6)
load = instrument_data[:, 1] * 25
temperature = instrument_data[:, 2] * 150
deformation = instrument_data[:, 3] * 0.5

# Convert frame number to time - data is collected at 10 Hz
instrument_data[:, 0] = instrument_data[:, 0] / 10

plt.plot(instrument_data[:, 0], temperature)
plt.xlabel("Time (s)")
plt.ylabel("Temperature (°C)")
plt.show()

plt.plot(instrument_data[:, 0], load)
plt.xlabel("Time (s)")
plt.ylabel("Load (N)")
plt.show()

plt.plot(instrument_data[:, 0], deformation)
plt.xlabel("Time (s)")
plt.ylabel("Sample deformation (mm)")
plt.show()

From this data we can see the sequence of the experiment, there is a heating stage, a steady state at high temperature, a deformation and a cooling stage.

We can now plot our fitted peak data along with the experimental data to see how the crystal structure changed during the experiment. Let's try and get the ratio of hexagonal to cubic crystals (alpha to beta phase). We can use the (10-11) maxima to represent the alpha phase and the (110) maxima to represent the beta phase.

In [ ]:
experiment.plot_fit_parameter("(0002)(110)(10-11)", "maximum_3_center", show_points=True)
experiment.plot_fit_parameter("(0002)(110)(10-11)", "maximum_3_amplitude", show_points=True)

We can see that the angle of the 10-11 peak shifted down with increasing temperature (due to the increase in lattice spacing with thermal energy) and decreased in amplitude - getting quite small during the high temperature section of the experiment.

In [ ]:
experiment.plot_fit_parameter('(0002)(110)(10-11)', "maximum_2_center", show_points=True) 
experiment.plot_fit_parameter('(0002)(110)(10-11)', "maximum_2_amplitude", show_points=True) 

The 110 peak, shows matching trends, the peak position shifts down a little and the amplitude greatly increases. Here we see the hexagonal $(\alpha)$ structure being transformed to the cubic $(\beta)$ structure.

To a first order approximation we could calculate the ratio of the $\alpha$ to $\beta$ phase by considering the ratio of peak amplitudes. This is not quite correct because there is also a decrease in peak amplitude with increasing two theta angle and the peaks dont have the same two theta angle. We would also probably like to take an average from multiple peaks too but this will give us a rough idea.

In [ ]:
alpha_amplitude = experiment.get_fit_parameter("(0002)(110)(10-11)", "maximum_1_amplitude")
beta_amplitude = experiment.get_fit_parameter("(0002)(110)(10-11)", "maximum_2_amplitude")
percentage_alpha = alpha_amplitude[:, 1] / (alpha_amplitude[:, 1] + beta_amplitude[:, 1])

import matplotlib.pyplot as plt
plt.plot(alpha_amplitude[:, 0], percentage_alpha, "-x")
plt.ylabel("Fraction alpha phase")
plt.xlabel("Time (s)")
plt.show()

Now lets put the phase ratio together with the instrument data to see the correlation.

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(alpha_amplitude[:, 0], percentage_alpha, "-x")
ax1.set_ylabel("Fraction alpha phase")
ax1.set_xlabel("Time (s)")

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax2.set_ylabel('Temperature (°C)')
ax2.plot(instrument_data[:, 0], temperature, color="green")
ax2.tick_params(axis='y')

fig.tight_layout() 
plt.show()

As expected we can see the change in crystal structure is strongly related to the temperature. We also want to know about the deformation though as we can see this distinct from the temperature change between 3000 and 400 seconds. We can plot the deformation, temperature and crystal ratio on the same plot to see them all together:

(This dual axis plot is taken from the example at: https://matplotlib.org/examples/pylab_examples/multiple_yaxis_with_spines.html)


In [ ]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

fig, host = plt.subplots()
fig.subplots_adjust(right=1)

par1 = host.twinx()
par2 = host.twinx()

par2.spines["right"].set_position(("axes", 1.2))
make_patch_spines_invisible(par2)
par2.spines["right"].set_visible(True)

p1, = host.plot(alpha_amplitude[:, 0], percentage_alpha, "-x", label="Fraction alpha phase")
p2, = par1.plot(instrument_data[:, 0], temperature, "-", label="Temperature", color="green")
p3, = par2.plot(instrument_data[:, 0], deformation, "-", label="Deformation", color="red")

host.set_xlim(250, 400)
host.set_ylim(0, 0.6)
par1.set_ylim(500, 1000)

host.set_xlabel("Time (s)")
host.set_ylabel("Fraction alpha phase")
par1.set_ylabel("Temperature (°C)")
par2.set_ylabel("Deformation (mm)")
host.yaxis.label.set_color(p1.get_color())
par1.yaxis.label.set_color(p2.get_color())
par2.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
host.tick_params(axis='y', colors=p1.get_color(), **tkw)
par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
par2.tick_params(axis='y', colors=p3.get_color(), **tkw)
host.tick_params(axis='x', **tkw)

host.axvline(x=3160, ymin=0, ymax=1, lw=1, ls="--", color="k")
host.axvline(x=3360, ymin=0, ymax=1, lw=1, ls="--", color="k")

lines = [p1, p2, p3]

host.legend(lines, [l.get_label() for l in lines], bbox_to_anchor=(2, 1.0))

plt.show()

The start and end of the deformation are marked by the dashed lines.
We can see that although there is some change in the crystal structure, it isn't as simple as the temperature relationship where hexagonal crystals are converted directly to cubic crystals.

We conclude the analysis here since this workbook isn't meant to be about the science as such, however this shows how the fitted parameters can be extracted to complete an analysis.